In [4]:
import itertools as it
import urllib
from io import BytesIO

In [5]:
from lxml import etree
import pandas as pd

In [6]:
html_parser = etree.HTMLParser()

In [7]:
def get_dom(html):
    return etree.fromstring(html, html_parser)


def get_indexed_data(dom):
    *index, data = zip(
        *it.chain(
            *filter(
                lambda x: x,
                (
                    [[element.tag, k, v] for k, v in element.attrib.items()]
                    for element in dom.iterdescendants()
                ),
            )
        )
    )
    index = pd.MultiIndex.from_tuples(
        list(zip(*index)), names=["tag name", "attribute"]
    )
    return pd.DataFrame(list(data), columns=["value"], index=index)

In [8]:
def get_df_from(url):
    with urllib.request.urlopen(url) as fh:
        b = BytesIO(fh.read())
    b.seek(0)
    return get_indexed_data(get_dom(b.read().decode("utf-8")))

## Jupyter notebook styling right aligns cells. Force left align.

In [9]:
%%html

<style>
   table {border-collapse:collapse; table-layout:fixed; width:350px;}
   table td {border:solid 1px #fab; width:110px; word-wrap:break-word; text-align:left !important;}
</style>

In [10]:
if __name__ == "__main__":
    from IPython.display import display, HTML

    display(HTML(get_df_from("https://twitter.com/").to_html()))